In [1]:
import pulp
import datagen as dgen
import organization as org
import itertools

def create_var_string(value):
    # Input in a 3-element-tuple
    return str(value[0])+'-'+str(value[1])+'-'+str(value[2])

# Get a list of teams
team_list = []
data = dgen.DataGen(2015)
conferences = data.league.conferences
for key in conferences.keys():
    conf = conferences[key]
    for div in conf.divisions.keys():
        teams = conf.divisions[div].teams
        for team in teams.keys():
            team_list.append(teams[team])
     

In [3]:
# Set number of games in the season
num_games = 82          
els = [tuple(x) for x in itertools.combinations(team_list, 2)]
# Create dictionary where keys are string representations of games
#    key: "{team1name}_{team2name}_{gamenum}"
#    value: information about home team and away team and game number
variable_dict = {}
for key in els:
    for idx in range(1,num_games+1):
        variable_dict[create_var_string((key[0].name,key[1].name,idx))] = {'home':key[0].name,'away':key[1].name,'homeDiv':key[0].division.name,\
                                                                           'homeConf':key[0].division.conference.name,\
                                                                           'awayConf':key[1].division.conference.name,\
                                                                           'awayDiv':key[1].division.name,'value':0,'gameNum':idx}
        variable_dict[create_var_string((key[1].name,key[0].name,idx))] = {'home':key[1].name,'away':key[0].name,'homeDiv':key[1].division.name,\
                                                                           'homeConf':key[1].division.conference.name,\
                                                                           'awayConf':key[0].division.conference.name,\
                                                                           'awayDiv':key[0].division.name,'value':0,'gameNum':idx}

x = pulp.LpVariable.dict("Game",variable_dict.keys(), lowBound=0, upBound=1, cat=pulp.LpInteger)
#Creating LP Problem
nba_schedule = pulp.LpProblem('NBA Schedule',pulp.LpMinimize)
for team in team_list:
    print("Adding conditions for ",team.name)
    nba_schedule += sum([x[key] for key in variable_dict.keys() if variable_dict[key]['home'] == team.name \
            or variable_dict[key]['away'] == team.name]) == num_games
    '''
    nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if (variable_dict[key]['home'] == team.name or\
                        variable_dict[key]['away'] == team.name) and \
                        variable_dict[key]['homeDiv'] == variable_dict[key]['awayDiv']]) == 16
    '''
    # Adding constraints for away and home games in a division
    for team_other in team_list:
        if team_other.name != team.name:
            nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['homeDiv'] == variable_dict[key]['awayDiv'] and\
                        variable_dict[key]['away'] == team.name and \
                             variable_dict[key]['home'] == team_other.name]) == 2
            nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['homeDiv'] == variable_dict[key]['awayDiv'] and\
                        variable_dict[key]['home'] == team.name and\
                            variable_dict[key]['away'] == team_other.name]) == 2
            #Out of conference teams
            nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['homeConf'] != variable_dict[key]['awayConf'] and\
                        variable_dict[key]['away'] == team.name and \
                             variable_dict[key]['home'] == team_other.name]) == 1
            nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['homeConf'] != variable_dict[key]['awayConf'] and\
                        variable_dict[key]['home'] == team.name and\
                            variable_dict[key]['away'] == team_other.name]) == 1
    # Ensuring one team plays only once on a single matchday       
    for num in range(1,num_games+1):
        nba_schedule += sum([x[key] for key in variable_dict.keys()\
                            if (variable_dict[key]['home'] == team.name or\
                                variable_dict[key]['away'] == team.name) and \
                            variable_dict[key]['gameNum'] == num]) == 1
    
    # Adding constraints for playing 4 games against the 6 selected out-of-division in-conference opponents
    
    teams_in_conf = set(team.division.conference.teams())
    assert(len(teams_in_conf) == 15)
    teams_in_div = set(team.division.teams.values())
    assert(len(teams_in_div) == 5)
    for team_other in team.conf_opponents:
        
        nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['away'] == team.name and \
                             variable_dict[key]['home'] == team_other.name]) == 2
        nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['home'] == team.name and \
                             variable_dict[key]['away'] == team_other.name]) == 2
    
    # Adding constraints 3 games against the 4 remaining out-of-division in-conference teams    
    remaining_in_conf_teams = teams_in_conf - teams_in_div - team.conf_opponents
    assert len(remaining_in_conf_teams) == 4
    for team_other in remaining_in_conf_teams:
        nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['away'] == team.name and \
                             variable_dict[key]['home'] == team_other.name]) <= 2
        nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if ((variable_dict[key]['away'] == team.name and \
                             variable_dict[key]['home'] == team_other.name) or (variable_dict[key]['home'] == team.name and \
                             variable_dict[key]['away'] == team_other.name))]) == 3

('Adding conditions for ', 'Thunder')
('Adding conditions for ', 'Jazz')
('Adding conditions for ', 'Timberwolves')
('Adding conditions for ', 'Nuggets')
('Adding conditions for ', 'Trailblazers')
('Adding conditions for ', 'Warriors')
('Adding conditions for ', 'Kings')
('Adding conditions for ', 'Lakers')
('Adding conditions for ', 'Clippers')
('Adding conditions for ', 'Suns')
('Adding conditions for ', 'Rockets')
('Adding conditions for ', 'Grizzlies')
('Adding conditions for ', 'Mavericks')
('Adding conditions for ', 'Spurs')
('Adding conditions for ', 'Pelicans')
('Adding conditions for ', 'Bucks')
('Adding conditions for ', 'Pistons')
('Adding conditions for ', 'Pacers')
('Adding conditions for ', 'Bulls')
('Adding conditions for ', 'Cavaliers')
('Adding conditions for ', 'Celtics')
('Adding conditions for ', 'Nets')
('Adding conditions for ', '76ers')
('Adding conditions for ', 'Raptors')
('Adding conditions for ', 'Knicks')
('Adding conditions for ', 'Magic')
('Adding conditio

In [ ]:

'''            
for key in variable_dict.keys():
    home = variable_dict[key]['home']
    away = variable_dict[key]['away']
    game_no = variable_dict[key]['gameNum']
    rev_key = create_var_string((away,home,game_no))
    nba_schedule += sum([x[key]+x[rev_key]]) <= 1

        for gameNum in range(1,num_games+1):    
            nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['homeDiv'] == variable_dict[key]['awayDiv'] and\
                        variable_dict[key]['home'] == team.name and\
                            variable_dict[key]['away'] == team_other.name\
                                and variable_dict[key]['gameNum'] == gameNum]) <= 1
'''

In [4]:
print("Solving")
nba_schedule.solve()

Solving


1

In [9]:
#Saving the LP object
nba_schedule.writeLP("nba_schedule.lp")

In [7]:
list_celtics = []
for key in variable_dict.keys():
    if (variable_dict[key]['home'] == 'Celtics' or variable_dict[key]['away'] == 'Celtics') and x[key].value() != 0:
         list_celtics.append((key,x[key].value(),variable_dict[key]['gameNum']))

In [8]:
sorted(list_celtics,key=lambda x: x[2])

[('Warriors-Celtics-1', 1.0, 1),
 ('Celtics-Trailblazers-2', 1.0, 2),
 ('Celtics-Thunder-3', 1.0, 3),
 ('Nets-Celtics-4', 1.0, 4),
 ('Celtics-Bucks-5', 1.0, 5),
 ('Knicks-Celtics-6', 1.0, 6),
 ('Celtics-Lakers-7', 1.0, 7),
 ('Celtics-Magic-8', 1.0, 8),
 ('Celtics-Heat-9', 1.0, 9),
 ('Kings-Celtics-10', 1.0, 10),
 ('Nuggets-Celtics-11', 1.0, 11),
 ('Celtics-Grizzlies-12', 1.0, 12),
 ('Pistons-Celtics-13', 1.0, 13),
 ('Celtics-Nets-14', 1.0, 14),
 ('Celtics-Timberwolves-15', 1.0, 15),
 ('Raptors-Celtics-16', 1.0, 16),
 ('Timberwolves-Celtics-17', 1.0, 17),
 ('Celtics-Raptors-18', 1.0, 18),
 ('Celtics-Pacers-19', 1.0, 19),
 ('Suns-Celtics-20', 1.0, 20),
 ('Celtics-Suns-21', 1.0, 21),
 ('Celtics-Pistons-22', 1.0, 22),
 ('Knicks-Celtics-23', 1.0, 23),
 ('Mavericks-Celtics-24', 1.0, 24),
 ('Celtics-Warriors-25', 1.0, 25),
 ('Bulls-Celtics-26', 1.0, 26),
 ('Celtics-Pistons-27', 1.0, 27),
 ('Celtics-Hornets-28', 1.0, 28),
 ('Celtics-Rockets-29', 1.0, 29),
 ('Celtics-Cavaliers-30', 1.0, 30),
 (

###Testing on a smaller subset

In [185]:
league = {'A1':'A','A2':'A','A3':'A','A4':'A','B1':'B','B2':'B','B3':'B','B4':'B'}
els = [tuple(x) for x in itertools.combinations(league.keys(), 2)]
num_games = 12
team_list = league.keys()
variable_dict = {}
for key in els:
    for idx in range(1,num_games+1):
        variable_dict[create_var_string((key[0],key[1],idx))] = {'home':key[0],'away':key[1],'homeDiv':league[key[0]],\
                                                                           'awayDiv':league[key[1]],'value':0,'gameNum':idx}
        variable_dict[create_var_string((key[1],key[0],idx))] = {'home':key[1],'away':key[0],'homeDiv':league[key[1]],\
                                                                           'awayDiv':league[key[0]],'value':0,'gameNum':idx}

In [190]:
x = pulp.LpVariable.dict("Game",variable_dict.keys(), lowBound=0, upBound=1, cat=pulp.LpInteger)
#Creating LP Problem
nba_schedule = pulp.LpProblem('NBA Schedule',pulp.LpMinimize)
for team in team_list:
    print("Adding conditions for ",team)
    nba_schedule += sum([x[key] for key in variable_dict.keys() if variable_dict[key]['home'] == team \
            or variable_dict[key]['away'] == team]) == num_games
    for team_other in team_list:
        if team_other != team:
            nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['homeDiv'] == variable_dict[key]['awayDiv'] and\
                        variable_dict[key]['away'] == team and \
                             variable_dict[key]['home'] == team_other]) == 2
            
            nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['homeDiv'] == variable_dict[key]['awayDiv'] and\
                        variable_dict[key]['home'] == team and\
                            variable_dict[key]['away'] == team_other]) == 2
            
            
    for num in range(1,num_games+1):
        nba_schedule += sum([x[key] for key in variable_dict.keys()\
                            if (variable_dict[key]['home'] == team or variable_dict[key]['away'] == team) and \
                            variable_dict[key]['gameNum'] == num]) == 1

('Adding conditions for ', 'A1')
('Adding conditions for ', 'B4')
('Adding conditions for ', 'A3')
('Adding conditions for ', 'A2')
('Adding conditions for ', 'A4')
('Adding conditions for ', 'B2')
('Adding conditions for ', 'B3')
('Adding conditions for ', 'B1')


In [192]:
nba_schedule.solve()

1

In [193]:
for key in variable_dict.keys():
    if (variable_dict[key]['home'] == 'A1' or variable_dict[key]['away'] == 'A1') and x[key].value() != 0:
        print (key,x[key].value())

('A1-A3-11', 1.0)
('A4-A1-9', 1.0)
('A3-A1-8', 1.0)
('A2-A1-1', 1.0)
('A2-A1-6', 1.0)
('A1-A4-2', 1.0)
('A1-A3-4', 1.0)
('A4-A1-10', 1.0)
('A3-A1-12', 1.0)
('A1-A2-5', 1.0)
('A1-A2-3', 1.0)
('A1-A4-7', 1.0)
